In [1]:
import json
import mysql.connector
from mysql.connector import Error
import os

def extract_and_save_peer_evaluation_scores():
    """JSON 파일들에서 peer_evaluation_score를 추출하여 MariaDB에 저장"""
    
    # MariaDB 연결 설정
    db_config = {
        'host': '13.209.110.151',
        'port': 3306,
        'user': 'root',
        'password': 'root',
        'database': 'skala'
    }
    
    # JSON 파일 경로들
    json_files = [
        r'D:\Github\ai-server\app\personal_quarter_reports\agents\peer_evaluation_results_2024Q1_v2.json',
        r'D:\Github\ai-server\app\personal_quarter_reports\agents\peer_evaluation_results_2024Q2_v2.json',
        r'D:\Github\ai-server\app\personal_quarter_reports\agents\peer_evaluation_results_2024Q3_v2.json',
        r'D:\Github\ai-server\app\personal_quarter_reports\agents\peer_evaluation_results_2024Q4_v2.json'
    ]
    
    try:
        # 데이터베이스 연결
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor()
        
        for file_path in json_files:
            try:
                # JSON 파일 읽기
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                
                # 데이터 검증 및 추출
                if data.get('success') and data.get('data'):
                    eval_data = data['data']
                    user_id = eval_data.get('user_id')
                    year = eval_data.get('year')
                    quarter = eval_data.get('quarter')
                    peer_evaluation_score = eval_data.get('peer_evaluation_score')
                    
                    print(f"처리 중: {year}Q{quarter} - User {user_id} - Score: {peer_evaluation_score}")
                    
                    # user_final_scores 테이블의 peer_score 컬럼 업데이트
                    # 테이블 컬럼명: evaluation_year, evaluation_quarter 사용
                    update_query = """
                        UPDATE user_final_scores 
                        SET peer_score = %s, updated_at = NOW() 
                        WHERE user_id = %s AND evaluation_year = %s AND evaluation_quarter = %s
                    """
                    
                    cursor.execute(update_query, (peer_evaluation_score, user_id, year, quarter))
                    
                    # 업데이트된 행이 없으면 새로 INSERT
                    if cursor.rowcount == 0:
                        insert_query = """
                            INSERT INTO user_final_scores (user_id, evaluation_year, evaluation_quarter, peer_score, created_at, updated_at)
                            VALUES (%s, %s, %s, %s, NOW(), NOW())
                        """
                        cursor.execute(insert_query, (user_id, year, quarter, peer_evaluation_score))
                        print(f"새 레코드 생성: {year}Q{quarter} - User {user_id}")
                    
                    connection.commit()
                    
                    print(f"저장 완료: {year}Q{quarter}")
                    
                else:
                    print(f"데이터 형식 오류: {file_path}")
                    
            except FileNotFoundError:
                print(f"파일을 찾을 수 없습니다: {file_path}")
            except json.JSONDecodeError:
                print(f"JSON 파싱 오류: {file_path}")
            except Exception as e:
                print(f"파일 처리 오류 ({file_path}): {str(e)}")
        
        print("모든 데이터 처리 완료")
        
    except Error as e:
        print(f"데이터베이스 오류: {e}")
    
    finally:
        # 연결 종료
        if connection.is_connected():
            cursor.close()
            connection.close()

def verify_data():
    """저장된 데이터 확인"""
    db_config = {
        'host': '13.209.110.151',
        'port': 3306,
        'user': 'root',
        'password': 'root',
        'database': 'skala'
    }
    
    try:
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor()
        
        # 저장된 데이터 조회
        cursor.execute("""
            SELECT user_id, evaluation_year, evaluation_quarter, peer_score, updated_at 
            FROM user_final_scores 
            WHERE peer_score IS NOT NULL
            ORDER BY user_id, evaluation_year, evaluation_quarter
        """)
        
        results = cursor.fetchall()
        
        print("\n저장된 데이터:")
        print("User ID | Year | Quarter | Peer Score | Updated At")
        print("-" * 55)
        for row in results:
            print(f"{row[0]:7} | {row[1]:4} | {row[2]:7} | {row[3]:10} | {row[4]}")
            
    except Error as e:
        print(f"데이터베이스 오류: {e}")
    
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

if __name__ == "__main__":
    # 데이터 추출 및 저장
    extract_and_save_peer_evaluation_scores()
    
    # 저장된 데이터 확인
    verify_data()

데이터 형식 오류: D:\Github\ai-server\app\personal_quarter_reports\agents\peer_evaluation_results_2024Q1_v2.json
데이터 형식 오류: D:\Github\ai-server\app\personal_quarter_reports\agents\peer_evaluation_results_2024Q2_v2.json
데이터 형식 오류: D:\Github\ai-server\app\personal_quarter_reports\agents\peer_evaluation_results_2024Q3_v2.json
데이터 형식 오류: D:\Github\ai-server\app\personal_quarter_reports\agents\peer_evaluation_results_2024Q4_v2.json
모든 데이터 처리 완료

저장된 데이터:
User ID | Year | Quarter | Peer Score | Updated At
-------------------------------------------------------
